# Universal estimator

Let $f(x|d_1,...,d_n)$ be a function, that for any fixed values of the parameters $d_i$, reduces to a PDF of x; $f$ thus is a family of functions, e.g., *log normal*.

$estimator(f, sample)$ is a function which learns the parameters $d_i$ of $f$ from a single input *sample* ($m$ observations drawn using $f$).

## Method
We assume the bounds on the parameters $d_i$ are known (e.g., $0 < d_1 < 2$, $d_2 >= 0$).

> 1. Generate synthetic *samples* using $f(x|d_1,...,d_n)$ where $d_i$ are drawn from $range(d_i)$.
> 2. Learn a DNN model from the synthetic data.
> 3. Predict the parameters $d_i$ on the input *sample*.

## Parameter range adjustment
The *estimator* is allowed to assume that the range of the parameter values is (0,1).

For that we use *range adjustment*:
> Input:
> 1. A funation $f(x|d_i)$ as defined above
> 2. Bounds on the parameters $d_i$
>
> Output: A function $g(x|t_i)$, which is the same as $f$, except that $t_i$ are in the range (0,1), 
> and such that $t$ is “typically” around 0, for some definition of typically.
>
> Let: $h(d)$ be a mapping function from $range(d)$ to the range (0,1) ($h$ is continuous within $range(d)$).
>
> Thus, $h^{-1}(t)$ is a mapping function from the range (0,1) to $range(d)$.
>
> Let: $g(x|t_i) = f(x|h^{-1}(t_i))$ where $h^{-1}(t_i)$ is a inverse of the *mapping* of the bounds on the parameters $d_i$.

### Range adjustment using $logit(x)$
> The *logit* is the inverse of the standard *logistic* function: $𝜎(x) = 1 / (1 + e^{-x})$ for $x ∈ (-∞, ∞)$
> <img src="images/logistic.png" />
> The *logit* function is defined as: $logit(x) = 𝜎^{-1}(x) = ln( x / (1-x) )$ for $x ∈ (0,1)$
> <img src="images/logit.png" />

We can use the *logistic* function and its *logit* inverse as follows:
> Let: $h(d) = logit(d)$ for $d ∈ (0,1)$
>
> Than $h^{-1}(t) = 𝜎(t)$ for $t ∈ (-∞, ∞)$

Define a function to return another function, i.e., an adjuster of a parameter from the range (0, 1) to the original range that a function takes.

```python
def adjuster_logit(low, high):
    # adjuster is defined in (-INF, INF)
    # low: parameter lower bound
    # high: parameter higher bound
    # return: a function that maps it's parameter (x) to the original range (low, high).

    LOW = 0 if math.inf == low else logistic(low)
    HIGH = 1 if math.inf == high else logistic(high)

    def adjust(x):
        # adjust is defined in (0, 1)
        return logit(LOW + x * (HIGH - LOW))
    
    return adjust
    
```

### Range adjustment using $arctan(x)$
> <img src="images/tan.png" />
> <img src="images/arctan.png" />

```python
def adjuster_arctan(low, high):
    # adjuster is defined in (-INF, INF)
    # low: parameter lower bound
    # high: parameter higher bound
    # return: a function defined in (0,1) that maps it's parameter (x) to the original range (low,high).

    LOW = -pi/2 if -INF == low else arctan(low)
    HIGH = pi/2 if INF == high else arctan(high)

    def adjust(x):
        # adjust is defined in (0, 1)
        if x < 0: return -INF
        if x > 1: return INF
        return tan(LOW + x * (HIGH - LOW))
    
    return adjust
```

### Range adjustment using $logit(x)$

In [1]:
import math
# from scipy.special import expit, logit

def expit(x):
    # expit(x) = 1 / (1 + e^(-x))
    # defined in (-INF, INF)
    return 1 / (1 + math.exp(-x))

def logit(p):
    # logit(p) = inv(expit) = ln(p/(1-p))
    # defined in (0, 1)
    if 0 == p: return -math.inf
    if 1 == p: return math.inf
    return math.log(p/(1-p))

def adjuster_logit(low, high):
    # adjuster is defined in (-INF, INF)
    # low: parameter lower bound
    # high: parameter higher bound
    # return: a function defined in (0,1) that maps it's parameter (x) to the original range (low,high).

    LOW = 0 if -math.inf == low else expit(low)
    HIGH = 1 if math.inf == high else expit(high)

    def adjust(x):
        # adjust is defined in (0, 1)
        if x < 0: return -math.inf
        if x > 1: return math.inf
        return logit(LOW + x * (HIGH - LOW))
    
    return adjust

### Range adjustment using $arctan(x)$

In [2]:
def adjuster_arctan(low, high):
    # adjuster is defined in (-INF, INF)
    # low: parameter lower bound
    # high: parameter higher bound
    # return: a function defined in (0,1) that maps it's parameter (x) to the original range (low,high).

    LOW = -math.pi/2 if -math.inf == low else math.atan(low)
    HIGH = math.pi/2 if math.inf == high else math.atan(high)

    def adjust(x):
        # adjust is defined in (0, 1)
        if x < 0: return -math.inf
        if x > 1: return math.inf
        return math.tan(LOW + x * (HIGH - LOW))
    
    return adjust

In [3]:
# test
print(adjuster_logit(low=-2, high=10)(0))
print(adjuster_arctan(low=-2, high=10)(0))
print(adjuster_logit(low=-2, high=10)(1))
print(adjuster_arctan(low=-2, high=10)(1))

-2.0
-1.9999999999999996
10.00000000000097
10.00000000000001


In [ ]:
from scipy import stats
from scipy.stats import lognorm

def lognormal_sample(config, size):
    return lognorm.rvs(s=config[0], loc=config[1], size=size, random_state=RANDOM_STATE)

def lognormal_next_config(param_space):

    """
    return a (uniform/normal) random parameters within param_space
    """

    # random alpha from param_space(alpha)
    alpha_low = param_space[0][0]
    alpha_high = param_space[0][1]
    adjust = adjuster_logit(low=alpha_low, high=alpha_high)
    x = np.random.uniform(0.0, 1.0, size=1)[0]
    alpha = adjust(x)

    # random loc from param_space(loc)
    loc_low = param_space[1][0]
    loc_high = param_space[1][1]
    adjust_loc = adjuster_logit(low=loc_low, high=loc_high)
    x = np.random.uniform(0.0, 1.0, size=1)[0]
    loc = adjust_loc(x)
    
    return alpha, loc
